In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

%matplotlib inline

In [2]:
inp_df = pd.read_csv(r'TimeStamp_clients.csv')

Preprocessing data - deleted unnecessary columns to reduce noise


In [3]:
inp_df = inp_df.drop(['Suffix'], axis = 1)
inp_df = inp_df.drop(['Zip'], axis = 1)
inp_df = inp_df.drop(['Birthdate'], axis = 1)
inp_df = inp_df.drop(['Gender'], axis = 1)
inp_df = inp_df.drop(['Race'], axis = 1)
inp_df = inp_df.drop(['Creatingagency'], axis = 1)
inp_df = inp_df.drop(['Visitdt'], axis = 1)
inp_df = inp_df.drop(['Visittm'], axis = 1)
inp_df["Fullname"] = inp_df["Firstname"] +' '+inp_df["Lastname"]
inp_df = inp_df.drop(['Lastname'], axis = 1)
inp_df = inp_df.drop(['Firstname'], axis = 1)
temp = inp_df['Scandatetime'].str.split(' ')
inp_df['Scandate'] = temp.str.get(0)
inp_df = inp_df.drop(['Scandatetime'], axis = 1)
inp_df = inp_df[inp_df['Entrydate'].notnull()]
inp_df['Entrydate'] = pd.to_datetime(inp_df['Entrydate'])
inp_df['Scandate'] = pd.to_datetime(inp_df['Scandate'])
inp_df['Difference'] = inp_df.groupby(['Fullname'])['Scandate'].apply(lambda x: x.diff())
inp_df['Year'] = inp_df['Scandate'].dt.to_period('Y')



In [4]:
inp_df.head(6)

,Entrydate,Fullname,Scandate,Difference,Year
0,2017-12-20,Ivan Castro,2018-01-02,NaT,2018
1,2017-10-23,Carlos Ramos,2018-01-02,NaT,2018
2,2014-04-12,Richard Bennett,2018-01-02,NaT,2018
3,2017-06-25,Francis Lewis,2018-01-02,NaT,2018
4,2017-09-03,Daniel Deramus,2018-01-02,NaT,2018
5,2017-05-25,Rena Davis,2018-01-02,NaT,2018


In [13]:
inp_df[inp_df['Fullname'] == "Santos Morales"]

,Entrydate,Fullname,Scandate,Difference,Year
33166,2014-06-18,Santos Morales,2018-06-23,NaT,2018
33392,2014-06-18,Santos Morales,2018-06-24,1 days,2018
33690,2014-06-18,Santos Morales,2018-06-25,1 days,2018
33897,2014-06-18,Santos Morales,2018-06-26,1 days,2018
34908,2014-06-18,Santos Morales,2018-07-01,5 days,2018
35268,2014-06-18,Santos Morales,2018-07-02,1 days,2018
35692,2014-06-18,Santos Morales,2018-07-05,3 days,2018
35904,2014-06-18,Santos Morales,2018-07-06,1 days,2018
36357,2014-06-18,Santos Morales,2018-07-08,2 days,2018
36440,2014-06-18,Santos Morales,2018-07-09,1 days,2018


In [6]:
inp2_df = inp_df.groupby(['Fullname', 'Year']).size().reset_index(name='Size')

In [7]:
inp2_df = inp2_df.pivot(index = 'Fullname', columns = 'Year')
inp2_df.columns = inp2_df.columns.droplevel(0)
inp2_df = inp2_df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('.'))
inp2_df["Total"] = inp2_df["2018"] + inp2_df["2019"]

In [8]:
inp2_df.head()
#inp2_df.shape

Year,2018,2019,Total
Fullname,,,
Melissa Roberts,3.0,0.0,3.0
Timmon Wyane Hall,11.0,6.0,17.0
Vanessa Polanco,1.0,0.0,1.0
douglas swab,2.0,3.0,5.0
ivonna molina,0.0,2.0,2.0


In [9]:
inp2_df.describe()

Year,2018,2019,Total
count,11851.000000,11851.000000,11851.000000
mean,5.963294,3.609400,9.572694
std,12.658151,8.031325,19.410994
min,0.000000,0.000000,1.000000
25%,1.000000,0.000000,1.000000
50%,1.000000,1.000000,3.000000
75%,6.000000,3.000000,9.000000
max,200.000000,146.000000,315.000000


In [11]:
inp2_df = inp2_df.sort_values("Total",ascending=False)

In [12]:
inp2_df.head(10)

Year,2018,2019,Total
Fullname,,,
Christine Stamas,200.0,115.0,315.0
Hung Dang,154.0,99.0,253.0
Geoffrey Musazi,149.0,103.0,252.0
Vera Menchan,143.0,108.0,251.0
Sheila & Marco Alvarez,155.0,88.0,243.0
Willie Barber,118.0,122.0,240.0
Santos Morales,93.0,146.0,239.0
Riliere Louis-Jean,165.0,73.0,238.0
Andrea Belcher,103.0,128.0,231.0
